<a href="https://colab.research.google.com/github/Upendra4204/Upendra4204/blob/main/Air_quality_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving city_day.csv.zip to city_day.csv.zip


In [ ]:
import pandas as pd
data=pd.read_csv('city_day.csv.zip')
df=pd.DataFrame(data)
df.head()

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,2015-01-02,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,2015-01-03,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,2015-01-04,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,2015-01-05,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN


##The dataset includes various air quality measures such as PM2.5, PM10, NO, NO2, NOx, NH3, CO, SO2, O3, Benzene, Toluene, and Xylene across different cities and dates.

##The target variable for prediction appears to be the AQI (Air Quality Index), and AQI_Bucket could be a categorical representation of AQI.

In [ ]:
df_info = df.info()
df_describe = df.describe()
print('df_info:',df_info)
print('df_describe:',df_describe)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29531 entries, 0 to 29530
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        29531 non-null  object 
 1   Date        29531 non-null  object 
 2   PM2.5       24933 non-null  float64
 3   PM10        18391 non-null  float64
 4   NO          25949 non-null  float64
 5   NO2         25946 non-null  float64
 6   NOx         25346 non-null  float64
 7   NH3         19203 non-null  float64
 8   CO          27472 non-null  float64
 9   SO2         25677 non-null  float64
 10  O3          25509 non-null  float64
 11  Benzene     23908 non-null  float64
 12  Toluene     21490 non-null  float64
 13  Xylene      11422 non-null  float64
 14  AQI         24850 non-null  float64
 15  AQI_Bucket  24850 non-null  object 
dtypes: float64(13), object(3)
memory usage: 3.6+ MB
df_info: None
df_describe:               PM2.5          PM10            NO           NO2           N

## Basic statistical details provide insights into the distribution of numerical features, such as mean and standard deviation for pollutants and AQI.

## There are missing values in several columns, including PM2.5, PM10, and AQI, which will require handling.

In [ ]:
df_missing_values = df.isnull().sum()
df_missing_values

City              0
Date              0
PM2.5          4598
PM10          11140
NO             3582
NO2            3585
NOx            4185
NH3           10328
CO             2059
SO2            3854
O3             4022
Benzene        5623
Toluene        8041
Xylene        18109
AQI            4681
AQI_Bucket     4681
dtype: int64

## The dataset has missing values in several columns, with Xylene, PM10, and NH3 having the highest number of missing values.

In [ ]:
df = df.drop('Xylene', axis=1)
# Removing the 'Xylene' column due to a large number of missing values

KeyError: "['Xylene'] not found in axis"

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# Imputing missing values for numerical columns with their median
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
imputer = SimpleImputer(strategy='median')
df[num_cols] = imputer.fit_transform(df[num_cols])

In [ ]:
cat_cols = ['City', 'AQI_Bucket']
label_encoder = LabelEncoder()
for col in cat_cols:
    df[col] = label_encoder.fit_transform(df[col].astype(str))

print('Missing values handled and categorical variables encoded.')
print('Displaying the first few rows of the processed dataset:')
print(df.head())

Missing values handled and categorical variables encoded.
Displaying the first few rows of the processed dataset:
   City        Date  PM2.5   PM10     NO    NO2    NOx    NH3     CO    SO2  \
0     0  2015-01-01  48.57  95.68   0.92  18.22  17.15  15.85   0.92  27.64   
1     0  2015-01-02  48.57  95.68   0.97  15.69  16.46  15.85   0.97  24.55   
2     0  2015-01-03  48.57  95.68  17.40  19.30  29.70  15.85  17.40  29.07   
3     0  2015-01-04  48.57  95.68   1.70  18.48  17.97  15.85   1.70  18.59   
4     0  2015-01-05  48.57  95.68  22.10  21.42  37.76  15.85  22.10  39.33   

       O3  Benzene  Toluene    AQI  AQI_Bucket  
0  133.36     0.00     0.02  118.0           6  
1   34.06     3.68     5.50  118.0           6  
2   30.70     6.80    16.40  118.0           6  
3   36.08     4.43    10.14  118.0           6  
4   39.31     7.01    18.89  118.0           6  


## Missing values have been handled, and categorical variables encoded. The 'Xylene' column was removed due to a large number of missing values. The dataset is now ready for the next steps in the modeling process

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Dividing data into features (X) and target (Y)
X = df.drop(['AQI', 'AQI_Bucket', 'City', 'Date'], axis=1)
Y = df['AQI']

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Data divided into features and target, train-test split done, and features scaled

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Building the linear regression model
model = LinearRegression()
model.fit(X_train_scaled, Y_train)

# Predicting on the test set
Y_pred = model.predict(X_test_scaled)

# Evaluating the model
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 2966.232784231658
R^2 Score: 0.807318199960089


## The Linear Regression model has been evaluated, showing a Mean Squared Error (MSE) of 2966.23 and an R^2 Score of 0.807. The R^2 Score indicates that the model explains approximately 80.7% of the variance in the AQI, which is a strong initial performance for a baseline model.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

# Using a simpler approach with predefined parameters to avoid long computation times
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)

# Performing cross-validation
cv_scores = cross_val_score(model, X_train_scaled, Y_train, cv=5, scoring='neg_mean_squared_error')

# Calculating mean score
mean_cv_score = np.mean(cv_scores)

print('Mean CV Score (Neg Mean Squared Error):', mean_cv_score)

Mean CV Score (Neg Mean Squared Error): -2351.6611719778875


## The cross-validation process for the RandomForestRegressor model, using predefined parameters to streamline computation, resulted in a mean cross-validation score (negative mean squared error) of approximately -2351.66. This score indicates the average performance of the model across different subsets of the training data, providing a more generalized measure of its predictive accuracy.